In [1]:
import sys
sys.path.append("../../")
from simulation_classes.flying_cars import FlyingCars
from simulation_classes.simulation_periodic import SimulationPeriodic
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
"""
シミュレーションの基本的な設定
基本的には ./periodic_simulation.ipynbを参考にしている。
DRONE_NUMについてfor文を回すためのファイル
"""

TOTAL_TIME = 100
time_step = 0.2
simulation_steps = int(TOTAL_TIME / time_step)
scale_factor = 20
legal_speed = 1.5
real_legal_speed = min(2, legal_speed) * scale_factor * 3.6
TOTAL_DISTANCE = 100
boundary_condition = "PERIODIC"
UPDATE_RULE = "parallel"
MAX_ACC_REAL_WORLD = 2 # 0.2Gくらい
MIN_ACC_REAL_WORLD = 5 # 0.5Gくらい

# 先頭の車が前になるように置く(一番後ろの車はx=0でスタート)
def scatter_cars_around(total_distance, car_num, idx, should_scatter=False):
    delta = total_distance / car_num
    return total_distance - delta * (idx+1)

In [3]:
result_obj = {}

min_headway = 20 # 一番密度が高い時の車間距離を [m] で設定
drone_num_for_max_density = int(TOTAL_DISTANCE * scale_factor / min_headway) #その時のドローン台数
print(f"最大ドローン台数：{drone_num_for_max_density}台")
increment = 5

最大ドローン台数：100台


### ここからシミュレーション開始。

In [5]:
max_acc = MAX_ACC_REAL_WORLD / scale_factor # ルンゲクッタの時に加速度にdelta_tがかかることに注意
min_acc = MIN_ACC_REAL_WORLD / scale_factor

helly_params = {
    "max_accel": max_acc,
    "min_accel": min_acc,
    "lambda_1": 0.2,
    "lambda_2": 0.4,
    "d": 0.5,
    "T_des": 1,
  }

simulation_list = []

for i in range(int(drone_num_for_max_density / increment)):
    DRONE_NUM = (i+1) * increment
    if (DRONE_NUM >15):
        break
    cars_list = [FlyingCars(xcor=scatter_cars_around(TOTAL_DISTANCE, DRONE_NUM, i),
                     ycor=0, 
                     v_0=0,
                     legal_speed=legal_speed, 
                     scale_factor=scale_factor,
                     drone_idx = i,
                     helly_params=helly_params,
                     )
               for i in range(DRONE_NUM)]
    print(DRONE_NUM)
    simulation = SimulationPeriodic(
        TOTAL_TIME=TOTAL_TIME, 
        time_step=time_step, 
        scale_factor=scale_factor,
        drone_list=cars_list,
        TOTAL_DISTANCE=TOTAL_DISTANCE,
        UPDATE_RULE=UPDATE_RULE )
    simulation.run()
    simulation_list.append(simulation)

    

5
===PARALLEL実行===
FINISHED
10
===PARALLEL実行===
FINISHED
15
===PARALLEL実行===
FINISHED


### ここからは解析に入る

In [11]:
dataList = []
for simulation in simulation_list:
    cars = simulation.drone_list
    # 流量を計算する
    velocity_list = []
    for car in cars:
        snap_frame = int(simulation_steps * 0.95)
        velocity_list.append(car.v_xList[snap_frame])
    mean_velocity = np.mean(np.array(velocity_list))
    print(velocity_list)
    print(f"平均速度は{mean_velocity}")

    # 密度：1km当たりの台数
    rho = (len(cars) / TOTAL_DISTANCE * scale_factor) * 1000
    flow = rho * mean_velocity

    # プロット用に保存
    dataToSave = {
        "density":rho,
        "flow":flow,
        "mean_velocity":mean_velocity,
        "start_position":"unscattered",
    }

[1.5, 1.5, 1.5, 1.5, 1.5]
平均速度は1.5
[1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5]
平均速度は1.5
[1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5]
平均速度は1.5


In [ ]:
import pandas as pd

dataList = []
for simulation in simulation_list:
    cars = simulation.drone_list
    # 流量を計算する
    velocity_list = []
    for car in cars:
        snap_frame = int(simulation_steps * 0.95)
        velocity_list.append(car.v_xList[snap_frame])
    mean_velocity = np.mean(np.array(velocity_list))

    # 密度：1km当たりの台数
    rho = len(cars) / (TOTAL_DISTANCE * scale_factor)  * 1000
    flow = rho * mean_velocity

    # プロット用に保存
    dataToSave = {
        "density":rho,
        "flow":flow,
        "mean_velocity":mean_velocity,
        "start_position":"unscattered",
        "drone_num":len(cars),
        "model":"Helly"
    }
    dataList.append(dataToSave)

df = pd.DataFrame(dataList)
df.head()

# 折れ線グラフを描画
plt.plot(df["density"], df["flow"], marker='o')
plt.xlabel('Density')
plt.ylabel('Flow')
plt.title('Fundamental Diagram')
plt.grid(True)
plt.show()
